In [1]:
import pandas as pd
import numpy as np
import sys, os
from IPython.display import display

In [4]:
# Declare all variables as strings. Spaces must be replaced with '+', i.e., change 'John Smith' to 'John+Smith'.
# Define the lat, long of the location and the year
lat, lon, year = 36.210, -120.085, 2010
# You must request an NSRDB api key from the link above
api_key = 'VvROkeZZCwGSD5nJphV3QNsfDUEplzgwzPSw0Kj1'
# Set the attributes to extract (e.g., dhi, ghi, etc.), separated by commas.
attributes = 'ghi,dhi,dni,wind_speed,air_temperature,solar_zenith_angle'
# Choose year of data
year = '2010'
# Set leap year to true or false. True will return leap day data if present, false will not.
leap_year = 'false'
# Set time interval in minutes, i.e., '30' is half hour intervals. Valid intervals are 30 & 60.
interval = '30'
# Specify Coordinated Universal Time (UTC), 'true' will use UTC, 'false' will use the local time zone of the data.
# NOTE: In order to use the NSRDB data in SAM, you must specify UTC as 'false'. SAM requires the data to be in the
# local time zone.
utc = 'false'
# Your full name, use '+' instead of spaces.
your_name = 'Tamuz+Hod'
# Your reason for using the NSRDB.
reason_for_use = 'class+project'
# Your affiliation
your_affiliation = 'UCSD'
# Your email address
your_email = 'test@ucsd.edu'
# Please join our mailing list so we can keep you up-to-date on new developments.
mailing_list = 'true'

# Declare url string
url = 'http://developer.nrel.gov/api/solar/nsrdb_psm3_download.csv?wkt=POINT({lon}%20{lat})&names={year}&leap_day={leap}&interval={interval}&utc={utc}&full_name={name}&email={email}&affiliation={affiliation}&mailing_list={mailing_list}&reason={reason}&api_key={api}&attributes={attr}'.format(year=year, lat=lat, lon=lon, leap=leap_year, interval=interval, utc=utc, name=your_name, email=your_email, mailing_list=mailing_list, affiliation=your_affiliation, reason=reason_for_use, api=api_key, attr=attributes)
# Return just the first 2 lines to get metadata:
info = pd.read_csv(url, nrows=1)
# See metadata for specified properties, e.g., timezone and elevation
timezone, elevation = info['Local Time Zone'], info['Elevation']

In [5]:
# View metadata
info

,Source,Location ID,City,State,Country,Latitude,Longitude,Time Zone,Elevation,Local Time Zone,...,Cloud Type 11,Cloud Type 12,Fill Flag 0,Fill Flag 1,Fill Flag 2,Fill Flag 3,Fill Flag 4,Fill Flag 5,Surface Albedo Units,Version
0,NSRDB,105434,-,-,-,36.21,-120.1,-8,102,-8,...,Dust,Smoke,NaN,Missing Image,Low Irradiance,Exceeds Clearsky,Missing CLoud Properties,Rayleigh Violation,NaN,v3.0.1


In [9]:
# Return all but first 2 lines of csv to get data:
df = pd.read_csv('http://developer.nrel.gov/api/solar/nsrdb_psm3_download.csv?wkt=POINT({lon}%20{lat})&names={year}&leap_day={leap}&interval={interval}&utc={utc}&full_name={name}&email={email}&affiliation={affiliation}&mailing_list={mailing_list}&reason={reason}&api_key={api}&attributes={attr}'.format(year=year, lat=lat, lon=lon, leap=leap_year, interval=interval, utc=utc, name=your_name, email=your_email, mailing_list=mailing_list, affiliation=your_affiliation, reason=reason_for_use, api=api_key, attr=attributes), skiprows=2)

# Set the time index in the pandas dataframe:
df = df.set_index(pd.date_range('1/1/{yr}'.format(yr=year), freq=interval+'Min', periods=525600/int(interval)))



In [13]:
# take a look
print('shape:',df.shape)
df.head(50)

shape: (17520, 11)


,Year,Month,Day,Hour,Minute,GHI,DHI,DNI,Wind Speed,Temperature,Solar Zenith Angle
2010-01-01 00:00:00,2010,1,1,0,0,0,0,0,0.7,9,166.76
2010-01-01 00:30:00,2010,1,1,0,30,0,0,0,0.6,9,165.63
2010-01-01 01:00:00,2010,1,1,1,0,0,0,0,0.5,9,162.05
2010-01-01 01:30:00,2010,1,1,1,30,0,0,0,0.5,9,157.17
2010-01-01 02:00:00,2010,1,1,2,0,0,0,0,0.5,9,151.67
2010-01-01 02:30:00,2010,1,1,2,30,0,0,0,0.6,8,145.85
2010-01-01 03:00:00,2010,1,1,3,0,0,0,0,0.7,8,139.89
2010-01-01 03:30:00,2010,1,1,3,30,0,0,0,0.7,8,133.86
2010-01-01 04:00:00,2010,1,1,4,0,0,0,0,0.8,8,127.81
2010-01-01 04:30:00,2010,1,1,4,30,0,0,0,0.7,8,121.78
